# Testing 
(Mainly locally, I don't know a method to get python notebook on vader5)
## Checking whether the code interpreter can be called

In [49]:

import requests

base_url = 'http://localhost:8502/api/chatbot'

auth_key = "qA94VhroHMHFN55inWgfAAkt1WEmzQ4J" # Only for testing
auth_string = "&auth_key=" + auth_key

# user_input = "This is a test request for your basic functionality. Please respond with (200 Ok) and exit."
# user_input = "This is a test request for your capabilities of using the code_interpreter tool. Please use the code_interpreter tool to run the following code exactly and only once: \"print('Hello, World!', flush=True)\"."
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to use matplotlib to visualize a sine wave from -2pi to +2pi."
# user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\nimport matplotlib\nmatplotlib.use('agg')\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()\"."
# The below input doesn't use the corerct matplotlib backend, so it might do unexpected things.
user_input = "This is a test regarding your capabilities of using the code_interpreter tool and whether it supports matplotlib. Please use the code_interpreter tool to run the following code: \"import numpy as np\nimport matplotlib.pyplot as plt\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()\"."
url = base_url + '/streamresponse?input=' + user_input + auth_string # leaving out the thread_id spawns a new thread

response = requests.get(url, stream=True) # The response can be streamed or gotten all at once.
complete_response = [] # The stream gets consumed when streamed, we'll store it here.

# Note that this way to cunsume the response chunks it and those chunks are surprisingly small, cutting up the response.
# A better way would be `for delta in response.iter_content(chunk_size=1024):`
for delta in response:
    print(delta)
    complete_response.append(delta.decode("utf-8"))

b'{"variant":"ServerHint","content":"thread_id:lOM9BmlsSCptiQ4TEXqmRpwIfYp8blZB"}'
b'{"variant":"Code","content":["","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":["{\\"","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":["code","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":["\\":\\"","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":["import","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":[" numpy","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":[" as","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":[" np","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":["\\\\n","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":["import","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":[" matplotlib","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","content":[".pyplot","call_XDWaYqPecGoHQltQIvBSRWlO"]}'
b'{"variant":"Code","conten

In [50]:
# Some elements are cut off early if the chunk size is not specified or too small, so to reconstruct, we'll need to append all element that don't start with "{" to the previous element.
reconstructed_response = []
for i in range(len(complete_response)):
    if complete_response[i][0] == "{":
        reconstructed_response.append(complete_response[i])
    else:
        reconstructed_response[-1] += complete_response[i]

for delta in reconstructed_response:
    print(delta)

{"variant":"ServerHint","content":"thread_id:lOM9BmlsSCptiQ4TEXqmRpwIfYp8blZB"}
{"variant":"Code","content":["","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":["{\"","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":["code","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":["\":\"","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":["import","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":[" numpy","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":[" as","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":[" np","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":["\\n","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":["import","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":[" matplotlib","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":[".pyplot","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"variant":"Code","content":[" as","call_XDWaYqPecGoHQltQIvBSRWlO"]}
{"

In [51]:
# Now we can parse the response as JSON
import json
response_json = [json.loads(x) for x in reconstructed_response]
for element in response_json:
    print(element)

{'variant': 'ServerHint', 'content': 'thread_id:lOM9BmlsSCptiQ4TEXqmRpwIfYp8blZB'}
{'variant': 'Code', 'content': ['', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['{"', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['code', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['":"', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['import', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': [' numpy', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': [' as', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': [' np', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['\\n', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['import', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': [' matplotlib', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'content': ['.pyplot', 'call_XDWaYqPecGoHQltQIvBSRWlO']}
{'variant': 'Code', 'co

In [52]:
# We can use this to easily extract only the Assistant's responses
assistant_responses = [x for x in response_json if x["variant"] == "Assistant"]
print("".join([x["content"] for x in assistant_responses])) # This will print the Assistant's responses in order.

It seems that the code execution returned a warning indicating that the figure cannot be shown in the current environment due to non-interactivity. I will try to re-run the plotting command to see if it can yield an output.  Let's proceed with executing the code again.The sine wave plot has been generated and saved as an image. While it cannot be displayed directly here, I can provide you with the file path for downloading or further usage.

The plot is saved as "sine_wave_plot.png". If you would like to access or download this file, please let me know!


In [53]:
# Or the generated Code
code_responses = [x for x in response_json if x["variant"] == "Code"]
print("".join([x["content"][0] for x in code_responses])) # This will print the generated code in order.

{"code":"import numpy as np\nimport matplotlib.pyplot as plt\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.show()"}{"code":"import numpy as np\nimport matplotlib.pyplot as plt\nt = np.linspace(-2 * np.pi, 2 * np.pi, 100)\nsine_wave = np.sin(t)\nplt.figure(figsize=(10, 5))\nplt.plot(t, sine_wave, label='Sine Wave')\nplt.title('Sine Wave from -2π to 2π')\nplt.xlabel('Angle (radians)')\nplt.ylabel('Sine value')\nplt.axhline(0, color='black', linewidth=0.5, linestyle='--')\nplt.axvline(0, color='black', linewidth=0.5, linestyle='--')\nplt.grid()\nplt.legend()\nplt.savefig('/tmp/sine_wave_plot.png')\nplt.close()"}


In [54]:
# Or all coded output
code_output_responses = [x["content"][0] for x in response_json if x["variant"] == "CodeOutput"]
for error in code_output_responses:
    print(error)


<string>:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown





In [55]:
# Testing the get_thread endpoint
thread_id = "dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw" # for testing
url = base_url + '/getthread?thread_id=' + thread_id + auth_string
response = requests.get(url)
print(response.json())

[{'variant': 'ServerHint', 'content': 'thread_id:dAfvxCjCBayzHVMkZPW0TKkeIk4jtGaw'}, {'variant': 'User', 'content': 'This is a test regarding your capabilities of using the code_interpreter tool. Please call the code_interpreter tool with the input \\"2+2\\".'}, {'variant': 'Code', 'content': ['\\"', '\\']}, {'variant': 'CodeOutput', 'content': ['The code interpreter was successfully called, but is not yet implemented. The inputs to this function were: code_interpreter ; Some(\\"{\\\\\\"code\\\\\\"', '\\\\\\"2+2\\\\\\"}\\"). @FrevaGPT, you may tell the user that the tool call was called and interpreted, but not yet implemented.']}, {'variant': 'OpenAIError', 'content': 'Error getting response.'}, {'variant': 'StreamEnd', 'content': 'Stream ended abruptly'}]
